In [1]:
import sys
sys.path.append('/PKEBenchmark')
import pke
!pip install datasets
!python -m spacy download en_core_web_sm
for path in sys.path:
    print(path)
print(pke.__file__)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
C:\Users\Ashraff\PKE-OurDataset
C:\ProgramData\Anaconda3\python39.zip
C:\ProgramData\Anaconda3\DLLs
C:\ProgramData\Anaconda3\lib
C:\ProgramData\Anaconda3

C:\Users\Ashraff\AppData\Roaming\Python\Python39\site-packages
C:\ProgramData\Anaconda3\lib\site-packages
C:\ProgramData\Anaconda3\lib\site-packages\win32
C:\ProgramData\Anaconda3\lib\site-packages\win32\lib
C:\ProgramData\Anaconda3\lib\site-packages\Pythonwin
/PKEBenchmark
C:\Users\Ashraff\PKE-OurDataset\pke\__init__.py


In [2]:
import spacy
from spacy.tokenizer import _get_regex_pattern

nlp = spacy.load("en_core_web_sm")

# Tokenization fix for in-word hyphens (e.g. 'non-linear' would be kept 
# as one token instead of default spacy behavior of 'non', '-', 'linear')
# https://spacy.io/usage/linguistic-features#native-tokenizer-additions

from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [ ]:
from tqdm.notebook import tqdm
from datasets import load_dataset

# benchmark = "inspec"
# benchmark = "pake"
benchmark = "pakeNew"
# benchmark = "pake-m-2k"

# ayesha08/pake-m-2k


# dataset = load_dataset('ayesha08/'+benchmark)
dataset = load_dataset('foyez/'+benchmark)

# load the inspec dataset
# dataset = load_dataset("csv", data_files={'train': 'test.csv', 'test': 'test1.csv'}, column_names=["title", "abstract"])

# pre-process training and test splits
train = []
for sample in tqdm(dataset['train']):
    train.append(nlp(sample["title"]+". "+sample["abstract"]))
test = []
for sample in tqdm(dataset['test']):
    test.append(nlp(sample["title"]+". "+sample["abstract"]))

Using custom data configuration foyez--pakeNew-a9722485006b1613
Reusing dataset csv (C:\Users\Ashraff\.cache\huggingface\datasets\foyez___csv\foyez--pakeNew-a9722485006b1613\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
from pke import compute_document_frequency, compute_lda_model
from string import punctuation


# computing LDA and DF counts

compute_document_frequency(
    documents=train,
    output_file='data/{}.df.gz'.format(benchmark),
    language='en',              # language of the input files
    normalization='stemming',   # use porter stemmer
    stoplist=list(punctuation), # stoplist (punctuation marks)
    n=5                         # compute n-grams up to 5-grams
)

compute_lda_model(
    documents=train,
    output_file="data/{}.lda.pickle.gz".format(benchmark),
    n_topics=500,               # number of topics
    language='en',              # language of the input files
    stoplist=list(punctuation), # stoplist (punctuation marks)
    normalization='stemming'    # use porter stemmer
)

In [ ]:
from pke import load_document_frequency_file, load_lda_model

df = load_document_frequency_file(input_file='data/{}.df.gz'.format(benchmark))
lda_model = load_lda_model(input_file="data/{}.lda.pickle.gz".format(benchmark))

In [ ]:
from pke.unsupervised import *

from timeit import default_timer as timer

outputs = {}
elapsed_times = {}
for model in [TfIdf, FirstPhrases, YAKE, PAKE]:
    outputs[model.__name__] = []
    
    extractor = model()
    start = timer()
    for i, doc in enumerate(tqdm(test)):
        extractor.load_document(input=doc, language='en')
        extractor.grammar_selection(grammar="NP: {<ADJ>*<NOUN|PROPN>+}")
        if model.__name__ in ["TfIdf"]:
            extractor.candidate_weighting(df=df)
        elif model.__name__ in ["TopicalPageRank"]:
            extractor.candidate_weighting(lda_model=lda_model)
        else:
            extractor.candidate_weighting()
        outputs[model.__name__].append([u for u,v in extractor.get_n_best(n=10, stemming=True)])
    end = timer()
    elapsed_times[model.__name__] = end - start
    #print("Elapsed time for {}: {:.2f}s - {:.2f} it/s".format(model.__name__, end - start,  len(test) / (end - start))) 

In [ ]:
from nltk.stem.snowball import SnowballStemmer as Stemmer
import numpy as np
    
# populates the references list with stemmed keyphrases
references = []

for sample in tqdm(dataset['test']):
    sample_keyphrases = []
    for keyphrase in sample["keyphrases"].split(','):
        keyphrase = keyphrase.strip()
        print("*************************Keyphrase*************************")
        print(keyphrase)
    
        # tokenize keyphrase
        tokens = [token.text for token in nlp(keyphrase)]
        
        # normalize tokens using Porter's stemming
        stems = [Stemmer('porter').stem(tok.lower()) for tok in tokens]
        print("*************************Stemed*************************")
        print(stems)
        sample_keyphrases.append(" ".join(stems))
       
    print("*************************sample_keyphrases*************************")
    references.append(sample_keyphrases)
    print(sample_keyphrases)

def evaluate(top_N_keyphrases, references, cutoff=5):
    P = len(set(top_N_keyphrases[:cutoff]) & set(references)) / len(top_N_keyphrases[:cutoff])
    R = len(set(top_N_keyphrases[:cutoff]) & set(references)) / len(references)
    F = (2*P*R)/(P+R) if (P+R) > 0 else 0 
    return (P, R, F)

In [ ]:
print("## Benchmarking on {}".format(benchmark))
print("| Model | it/s |  F@5 | F@10 |")
print("| :---- | ----:| ---: | ---: |")

# loop through the models
for model in outputs:
    
    f_scores = []
    # compute the P, R, F scores for the model
    for cutoff in [5, 10]:
        scores = []
        for i, output in enumerate(outputs[model]):
            scores.append(evaluate(output, references[i], cutoff))

        # compute the average scores
        P, R, F = np.mean(scores, axis=0)
        f_scores.append(F)
        
    print("| {}  | {:.1f} | {:.2f} | {:.2f} |".format(model,  len(test)/ elapsed_times[model], f_scores[0]*100, f_scores[1]*100))


        # print out the performance of the model
        #print("{} at {} P: {:.3f} R: {:.3f} F: {:.3f}".format(model, cutoff, avg_scores[0], avg_scores[1], avg_scores[2]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
model_name = np.array([]) 
f_5 = np.array([]) 
f_10 = np.array([])
# loop through the models
for model in outputs:
    
    f_scores = []
    # compute the P, R, F scores for the model
    for cutoff in [5, 10]:
        scores = []
        for i, output in enumerate(outputs[model]):
            scores.append(evaluate(output, references[i], cutoff))

        # compute the average scores
        P, R, F = np.mean(scores, axis=0)
        f_scores.append(F)
        
    model_name = np.append(model_name, model) 
    f_5 = np.append(f_5, f_scores[0]*100) 
    f_10 = np.append(f_10, f_scores[1]*100) 

plt.bar(model_name, f_5)
plt.show()
print("F@5")
plt.bar(model_name, f_10)
plt.show()
print("F@10")